In [98]:
import torch

In [99]:
# 计算出来的Attention张量形状为(numWindows*B, num_heads, window_size*window_size, window_size*window_size)
coords_h = torch.arange(2)
coords_h

tensor([0, 1])

In [100]:
coords_w = torch.arange(2)
coords_w

tensor([0, 1])

In [101]:
# x1 ,y1 = torch.meshgrid(x,y)
# 输出的是两个tensor，size就是x.size * y.size（行数是x的个数，列数是y的个数）
coords = torch.meshgrid([coords_h, coords_w])
coords

(tensor([[0, 0],
         [1, 1]]),
 tensor([[0, 1],
         [0, 1]]))

In [102]:
coords = torch.stack(coords)
coords

tensor([[[0, 0],
         [1, 1]],

        [[0, 1],
         [0, 1]]])

In [103]:
coords_flatten = torch.flatten(coords, 1)
coords_flatten

tensor([[0, 0, 1, 1],
        [0, 1, 0, 1]])

In [105]:
# 在第三个shape处加上None的作用：每一行的元素一个一个按列排序，且每一行排列出来的元素为一个axis=1的向量

# 利用广播机制，分别在第一维，第二维，插入一个维度，进行广播相减，得到 2, wh*ww, wh*ww的张量
relative_coords_first = coords_flatten[:, :, None]  # 2, wh*ww, 1
relative_coords_first

tensor([[[0],
         [0],
         [1],
         [1]],

        [[0],
         [1],
         [0],
         [1]]])

In [106]:
relative_coords_second = coords_flatten[:, None, :]  # 2, 1, wh*ww
relative_coords_second

tensor([[[0, 0, 1, 1]],

        [[0, 1, 0, 1]]])

In [107]:
relative_coords = relative_coords_first - relative_coords_second  # 最终得到 2, wh*ww, wh*ww 形状的张量
relative_coords

tensor([[[ 0,  0, -1, -1],
         [ 0,  0, -1, -1],
         [ 1,  1,  0,  0],
         [ 1,  1,  0,  0]],

        [[ 0, -1,  0, -1],
         [ 1,  0,  1,  0],
         [ 0, -1,  0, -1],
         [ 1,  0,  1,  0]]])

In [108]:
relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
relative_coords

tensor([[[ 0,  0],
         [ 0, -1],
         [-1,  0],
         [-1, -1]],

        [[ 0,  1],
         [ 0,  0],
         [-1,  1],
         [-1,  0]],

        [[ 1,  0],
         [ 1, -1],
         [ 0,  0],
         [ 0, -1]],

        [[ 1,  1],
         [ 1,  0],
         [ 0,  1],
         [ 0,  0]]])

In [109]:
# 因为采取的是相减，所以得到的索引是从负数开始的，我们加上偏移量，让其从0开始
relative_coords[:, :, 0] += 2 - 1
relative_coords

tensor([[[ 1,  0],
         [ 1, -1],
         [ 0,  0],
         [ 0, -1]],

        [[ 1,  1],
         [ 1,  0],
         [ 0,  1],
         [ 0,  0]],

        [[ 2,  0],
         [ 2, -1],
         [ 1,  0],
         [ 1, -1]],

        [[ 2,  1],
         [ 2,  0],
         [ 1,  1],
         [ 1,  0]]])

In [110]:
relative_coords[:, :, 1] += 2 - 1
relative_coords

tensor([[[1, 1],
         [1, 0],
         [0, 1],
         [0, 0]],

        [[1, 2],
         [1, 1],
         [0, 2],
         [0, 1]],

        [[2, 1],
         [2, 0],
         [1, 1],
         [1, 0]],

        [[2, 2],
         [2, 1],
         [1, 2],
         [1, 1]]])